**Temat:** Analiza sentymentu w tekstach internetowych w oparciu o sieci typu Transformer

**Wprowadzenie:** Analiza sentymentu to technika przetwarzania języka naturalnego (NLP), która identyfikuje ton emocjonalny w tekście, klasyfikując go na pozytywny, negatywny lub neutralny. Wykorzystuje się ją do badania opinii klientów, monitorowania reputacji marki czy analizy treści mediów społecznościowych.

**Cel projektu:** Celem projektu jest opracowanie i implementacja modelu analizy sentymentu, który pozwoli na klasyfikację opinii użytkowników na podstawie tekstów pochodzących z Internetu. Należy przeanalizować dane tekstowe, przygotować odpowiedni model oraz zaprezentować wyniki analizy.

In [22]:
%pip install datasets transformers torch langdetect --quiet

Note: you may need to restart the kernel to use updated packages.


### Ładowanie danych

In [1]:
from datasets import load_dataset

ds = load_dataset("clapAI/MultiLingualSentiment")

/Users/mikolaj/Desktop/STUDIA/CDV STOPIEŃ II/I ROK/II SEMESTR/Uczenie głebokie w przetwarzaniu języka/PROJEKT/Multilingual-sentiment-analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'source', 'domain', 'language'],
        num_rows: 3147478
    })
    validation: Dataset({
        features: ['text', 'label', 'source', 'domain', 'language'],
        num_rows: 393435
    })
    test: Dataset({
        features: ['text', 'label', 'source', 'domain', 'language'],
        num_rows: 393436
    })
})


In [ ]:
languages = ds['train'].unique('language')
ds_types = ['train', 'validation', 'test']
drop_columns = ['source', 'domain']
ds = ds.remove_columns(drop_columns)
# Create dictionary to store datasets for each language
datasets = {}

# Split train, validation and test for each language
for lang in languages:
    datasets[lang] = {}
    for ds_type in ds_types:
        datasets[lang][ds_type] = ds[ds_type].filter(
            lambda batch: [x == lang for x in batch['language']],
            batched = True,
            num_proc=4
            )
        
        # Reduce dataset by 100 times
        rows_counter = datasets[lang][ds_type].num_rows
        new_num_rows = round(rows_counter*0.01)
        datasets[lang][ds_type] = datasets[lang][ds_type].shuffle(seed=42)
        datasets[lang][ds_type] = datasets[lang][ds_type].select(range(new_num_rows))

Filter (num_proc=4): 100%|██████████| 393436/393436 [00:00<00:00, 1134966.48 examples/s]


### Czyszczenie datasetów z błędnych języków

In [ ]:
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"
    
for ds_type in ds_types:
        for lang in languages:
            datasets['en'][ds_type] = datasets['en'][ds_type].filter(
                lambda batch: [detect_language(x) == 'en' for x in batch['text']],
                batched=True,
                num_proc=4
            )

Filter (num_proc=4): 100%|██████████| 1275/1275 [00:01<00:00, 661.27 examples/s]


In [26]:
datasets['en']['train'][0]

{'text': 'Not allowed to withdraw my funds, and was told I can only cash out if I deactivate or close the account. Well if u say so.',
 'label': 'negative',
 'language': 'en'}

### Tokenizacja

In [9]:
from transformers import BertTokenizer
from datasets import concatenate_datasets

In [30]:
languages_to_process = ['en', 'es', 'zh']
labels_id = {'negative': 0, 'neutral': 1, 'positive': 2}

# Convert labels to IDs
def convert_labels_to_ids(batch):
    batch['label_id'] = [labels_id[label] for label in batch['label']]
    return batch

# train ds
train_ds_list = [datasets[lang]['train'] for lang in languages_to_process]
# Concatenate datasets for selected languages
train_ds = concatenate_datasets(train_ds_list)
train_ds = train_ds.map(convert_labels_to_ids, batched=True, num_proc=4)
train_ds = train_ds.shuffle(seed=42)

# eval ds
eval_ds_list = [datasets[lang]['validation'] for lang in languages_to_process]
# Concatenate datasets for selected languages
eval_ds = concatenate_datasets(eval_ds_list)
eval_ds = eval_ds.map(convert_labels_to_ids, batched=True, num_proc=4)
eval_ds = eval_ds.shuffle(seed=42)

# test ds
test_ds_list = [datasets[lang]['test'] for lang in languages_to_process]
# Concatenate datasets for selected languages
test_ds = concatenate_datasets(test_ds_list)
test_ds = test_ds.map(convert_labels_to_ids, batched=True, num_proc=4)
test_ds = test_ds.shuffle(seed=42)

Map (num_proc=4): 100%|██████████| 1942/1942 [00:00<00:00, 6541.98 examples/s]


In [32]:
print(train_ds[0])

{'text': '很实用，点赞。。。:)', 'label': 'positive', 'language': 'zh', 'label_id': 2}


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_and_encode(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128,  # BERT max sequence length
        # return_tensors=None,  # Returns PyTorch tensors
        # return_special_tokens_mask=True
    )



tokenized_datasets = {}
for lang, dataset in example_datasets.items():
    print(f"Tokenizing {lang} dataset...")
    tokenized_datasets[lang] = dataset.map(
        tokenize_and_encode,
        batched=True,
        batch_size=1000,  # Increased batch size
        num_proc=4,       # Use multiple CPU cores
        remove_columns=['text', 'language']  # Remove original columns we don't need
    )
    
    # Convert label to tensor format
    tokenized_datasets[lang] = tokenized_datasets[lang].map(
        lambda x: {'labels': x['label']},
        remove_columns=['label']
    )

print("\nExample of tokenized text:")
print(tokenized_datasets[list(tokenized_datasets.keys())[0]][0])

Tokenizing en dataset...


Map: 100%|██████████| 1215709/1215709 [01:51<00:00, 10866.46 examples/s]


Tokenizing es dataset...


Map: 100%|██████████| 178434/178434 [00:16<00:00, 11038.80 examples/s]


Tokenizing fr dataset...


Map: 100%|██████████| 210298/210298 [00:19<00:00, 10751.26 examples/s]


Example of tokenized text:
{'source': 'https://www.kaggle.com/datasets/choonkhonng/malaysia-restaurant-review-datasets', 'domain': 'restaurant reviews ', 'input_ids': [101, 138, 15198, 26069, 10169, 15198, 18301, 119, 23002, 10124, 27949, 13096, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 